# Прогнозирование оттока клиентов

In [1]:
# imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Описание признаков

In [2]:
column_desc_path = 'data/Описание массива.csv'
column_desc = (pd.read_csv(column_desc_path, sep=';',
                           usecols=['Наименование поля', 'Описание поля', 'Описание значений'],
                           index_col='Наименование поля'))

column_desc.style.set_properties(width = '100em')

,Описание поля,Описание значений
Наименование поля,,
DATA_TYPE,Тип выборки,TRAIN - обучающая выборка TEST - тестовая выборка
POLICY_ID,ID полиса,nan
POLICY_BEGIN_MONTH,Месяц начала действия полиса,nan
POLICY_END_MONTH,Месяц окончания действия полиса,nan
POLICY_IS_RENEWED,Факт пролонгации полиса. Прогнозируемый параметр,Для тестовой выборки параметр обнулён
POLICY_SALES_CHANNEL,Канал продаж полиса,nan
POLICY_SALES_CHANNEL_GROUP,Группа каналов продаж (группировка для колонки - канал продаж полиса),nan
POLICY_BRANCH,Филиал продажи полиса,nan
POLICY_MIN_AGE,Минимальный возраст лиц допущенных к управлению по полису,nan


In [3]:
def object2category(df):
    df = df.copy()
    df.loc[:, df.dtypes == 'object'] = df.loc[:, df.dtypes == 'object'].astype('category')
    return df

In [4]:
data_path = 'data/Данные для задачи.txt'
data = (pd.read_csv(data_path, sep=';')
          .assign(DATA_TYPE=lambda df: df.DATA_TYPE.str.rstrip())
          .pipe(object2category)
          .set_index(keys=['DATA_TYPE', 'POLICY_ID'], drop=True))
data

POLICY_BEGIN_MONTH  POLICY_END_MONTH  POLICY_IS_RENEWED  \
DATA_TYPE POLICY_ID                                                            
TRAIN     1                           1                 1                  1   
          2                           1                 1                  1   
          3                           1                 1                  1   
          4                           1                 1                  1   
          5                           1                 1                  0   
...                                 ...               ...                ...   
          96601                       3                 3                  1   
          96602                       4                 3                  1   
          96603                       3                 3                  1   
          96604                       3                 3                  1   
          96605                       3                 3                  1   

                     POLICY_SALES_CHANNEL  POLICY_SALES_CHANNEL_GROUP  \
DATA_TYPE POLICY_ID                                                     
TRAIN     1                            39                           1   
          2                            50                           5   
          3                            52                           6   
          4                            50                           5   
          5                            52                           6   
...                                   ...                         ...   
          96601                        55                           6   
          96602                        50                           5   
          96603                        50                           5   
          96604                        52                           6   
          96605                        54                           6   

                       POLICY_BRANCH  POLICY_MIN_AGE  \
DATA_TYPE POLICY_ID                                    
TRAIN     1                   Москва              51   
          2                   Москва              35   
          3                   Москва              41   
          4                   Москва              36   
          5          Санкт-Петербург              42   
...                              ...             ...   
          96601               Москва              38   
          96602      Санкт-Петербург              30   
          96603               Москва              59   
          96604               Москва              41   
          96605               Москва              26   

                     POLICY_MIN_DRIVING_EXPERIENCE   VEHICLE_MAKE  \
DATA_TYPE POLICY_ID                                                 
TRAIN     1                                     12     Land Rover   
          2                                      7           Opel   
          3                                      6            Kia   
          4                                     12        Citroen   
          5                                      5        Renault   
...                                            ...            ...   
          96601                                  6           Ford   
          96602                                  6  Mercedes-Benz   
          96603                                 31           Opel   
          96604                                 19         Toyota   
          96605                                  7            BMW   

                          VEHICLE_MODEL  ...  POLICY_PRV_CLM_GLT_N  \
DATA_TYPE POLICY_ID                      ...                         
TRAIN     1                   Discovery  ...                     N   
          2                      Zafira  ...                     0   
          3                        Soul  ...                    1L   
          4                   C-Crosser  ...         

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 96605 entries, ('TRAIN', 1) to ('TRAIN', 96605)
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   POLICY_BEGIN_MONTH             96605 non-null  int64   
 1   POLICY_END_MONTH               96605 non-null  int64   
 2   POLICY_IS_RENEWED              96605 non-null  int64   
 3   POLICY_SALES_CHANNEL           96605 non-null  int64   
 4   POLICY_SALES_CHANNEL_GROUP     96605 non-null  int64   
 5   POLICY_BRANCH                  96605 non-null  category
 6   POLICY_MIN_AGE                 96605 non-null  int64   
 7   POLICY_MIN_DRIVING_EXPERIENCE  96605 non-null  int64   
 8   VEHICLE_MAKE                   96605 non-null  category
 9   VEHICLE_MODEL                  96605 non-null  category
 10  VEHICLE_ENGINE_POWER           96605 non-null  float64 
 11  VEHICLE_IN_CREDIT              96605 non-null  int64   
 12  VEHICLE_SU

Разделим датасет на тестовую, обучающую и валидационную выборки.

In [6]:
from sklearn.model_selection import train_test_split

test, train = data.loc['TEST'].copy(), data.loc['TRAIN'].copy()
train, validation = train_test_split(train, test_size=0.2)

## Определим минимально возможную точность предсказательной модели

In [7]:
train.POLICY_IS_RENEWED.value_counts(normalize=True)

1    0.624901
0    0.375099
Name: POLICY_IS_RENEWED, dtype: float64

В случае если модель в 100% случаев будет прогнозировать, что полис будет пролонгирован, будет достигнута точность предсказания 62.6% на тренировочных данных.

Установим эту величину как порог для отсечения моделей.

## Модель

In [8]:
from sklearn.tree import DecisionTreeClassifier
columns_getter = ((train.dtypes == 'int64') | (train.dtypes == 'float64')) & (train.columns != 'POLICY_IS_RENEWED')
x_train, y_train = train.loc[:, columns_getter], train.loc[:, 'POLICY_IS_RENEWED']
decision_tree = DecisionTreeClassifier(criterion='gini',  max_depth=10)
decision_tree = decision_tree.fit(x_train, y_train)

In [9]:
decision_tree.score(x_train, y_train)

0.7111182882519177

Оценим качество на валидационной выборке.

In [10]:
x_validation, y_validation = validation.loc[:, columns_getter], validation.loc[:, 'POLICY_IS_RENEWED']
decision_tree.score(x_validation, y_validation)

0.6804030487017181

## Прогноз на тестовых данных

In [11]:
def predict(df, model, features): 
    x = df.loc[:, columns_getter]
    pred = model.predict(x)
    return pred

def predict_prob(df, model, features):
    x = df.loc[:, columns_getter]
    pred = model.predict_proba(x)[:, 1]
    return pred

In [12]:
(test.assign(POLICY_IS_RENEWED=lambda df: predict(df, decision_tree, columns_getter),
             POLICY_IS_RENEWED_PROBABILITY=lambda df: predict_prob(df, decision_tree, columns_getter))
     .loc[:, ['POLICY_IS_RENEWED', 'POLICY_IS_RENEWED_PROBABILITY']])

,POLICY_IS_RENEWED,POLICY_IS_RENEWED_PROBABILITY
POLICY_ID,,
10,1,0.831136
12,1,0.584197
13,0,0.342857
22,1,0.541667
37,1,0.769432
...,...,...
96574,1,0.831136
96577,1,0.694542
96586,1,0.831136


Сохраним обученную модель

In [13]:
!pip install joblib
import os
import joblib

modelname = 'model/model.joblib.pkl'
os.makedirs('model', exist_ok=True)

model_data = {'model' : decision_tree, 
             'types' : train.reset_index().dtypes.to_dict(),
             'features' : columns_getter}

joblib.dump(model_data, modelname)

Defaulting to user installation because normal site-packages is not writeable


['model/model.joblib.pkl']

## Тестирование сервера

Что предосмотреть в сервере?
* Запрос на прогноз (входные данные в форматах json, csv -> выходные в соответсвующих форматах)
* Логгирование
* Обработка исключений

Проверим, что сервер принемает файлы в формате JSON и CSV и возвращает ответы в соответствующих форматах.

In [14]:
import aiohttp
from io import StringIO, BytesIO

# test data to byte string in csv format
data_csv  = test.to_csv(sep=';').encode('utf-8')

# test datta to byte string in json format
data_json = test.reset_index().to_json().encode('utf-8')

csv_prediction = None
json_prediction = None

async with aiohttp.ClientSession() as session:
    with aiohttp.MultipartWriter('multipart/form-data') as mpwriter:
        part = mpwriter.append(BytesIO(data_csv),
                                    {'CONTENT-TYPE': 'text/csv'})
        part.set_content_disposition('attachment', filename='test.csv')
        
        part = mpwriter.append(BytesIO(data_json),
                                    {'CONTENT-TYPE': 'application/json'})
        part.set_content_disposition('attachment', filename='test.json')
        
    async with session.post("http://0.0.0.0:9000/predict", data=mpwriter) as resp:
          print(resp.status)
          reader = aiohttp.MultipartReader.from_response(resp)
          async for part in reader:
                content_type = part.headers[aiohttp.hdrs.CONTENT_TYPE]
                partdata = await part.text()
                if content_type == 'application/json':
                    json_prediction = partdata
                elif content_type == 'text/csv':
                    csv_prediction = partdata

In [15]:
# create dataframe from recieved csv file
pd.read_csv(StringIO(csv_prediction), sep=';')

,POLICY_ID,POLICY_IS_RENEWED,POLICY_IS_RENEWED_PROBABILITY
0,10,1,0.831136
1,12,1,0.584197
2,13,0,0.342857
3,22,1,0.541667
4,37,1,0.769432
...,...,...,...
19193,96574,1,0.831136
19194,96577,1,0.694542
19195,96586,1,0.831136
19196,96595,1,0.831136


In [16]:
# create dataframe from recieved json file
pd.read_json(StringIO(json_prediction))

,POLICY_ID,POLICY_IS_RENEWED,POLICY_IS_RENEWED_PROBABILITY
0,10,1,0.831136
1,12,1,0.584197
2,13,0,0.342857
3,22,1,0.541667
4,37,1,0.769432
...,...,...,...
19193,96574,1,0.831136
19194,96577,1,0.694542
19195,96586,1,0.831136
19196,96595,1,0.831136


Проверим, что сервер отвечает на некорректные запросы, возвращая код ошибки 415.

In [30]:
async with aiohttp.ClientSession() as session:
    with aiohttp.MultipartWriter('multipart/form-data') as mpwriter:
        # file in unsupported format
        part = mpwriter.append(BytesIO(data_csv),
                                    {'CONTENT-TYPE': 'text/error'})
        part.set_content_disposition('attachment', filename='test.csv')
        
        # invalid json file
        part = mpwriter.append(BytesIO(b'sdfsdggsvsvsv'),
                                    {'CONTENT-TYPE': 'application/json'})
        part.set_content_disposition('attachment', filename='test.json')
        
    async with session.post("http://0.0.0.0:9000/predict", data=mpwriter) as resp:
          print(resp.status)

415
